Installed biosimspace 2024.4.0 dev via
added gromacs and ambertools via conda-forge


In [ ]:
import BioSimSpace as BSS

In [ ]:
lig_num = 1

In [ ]:
protein_xtal = BSS.IO.readPDB("inputs/3STD_prep_fix.pdb", pdb4amber=True)[0]

Change view settings to highlight the xtal waters

In [ ]:
view = BSS.Notebook.View(protein_xtal)

In [ ]:
view.system()

extract crystallographic waters

In [ ]:
protein = protein_xtal.extract(
    [atom.index() for atom in protein_xtal.search("not resname HOH").atoms()]
)
water = protein_xtal.extract(
    [atom.index() for atom in protein_xtal.search("resname HOH").atoms()]
)

For the rest of the setup we are ignoring the xtal waters. Later we can check if GCMC found buried sites. The site of interest near the ligand is dry in the input

In [ ]:
lig = BSS.IO.readMolecules(f"inputs/lig{lig_num}.sdf")[0]

In [ ]:
view = BSS.Notebook.View(lig1)

In [ ]:
view.system()

In [ ]:
protein = BSS.Parameters.ff14SB(protein, work_dir="tmp").getMolecule()

In [ ]:
lig = BSS.Parameters.openff_unconstrained_2_2_0(lig).getMolecule()

In [ ]:
complex = lig + protein

In [ ]:
complex_sol = BSS.Solvent.tip3p(
    molecule=complex, box=3 * [60 * BSS.Units.Length.angstrom]
)

In [ ]:
view = BSS.Notebook.View(complex_sol)

In [ ]:
view.system()

In [ ]:
protocol = BSS.Protocol.Minimisation()
process = BSS.Process.OpenMM(complex_sol, protocol, platform="CUDA")
process.start()
process.wait()
minimised = process.getSystem()

In [ ]:
protocol = BSS.Protocol.Equilibration(
    runtime=500 * BSS.Units.Time.picosecond,
    pressure=1 * BSS.Units.Pressure.atm,
    temperature=300 * BSS.Units.Temperature.kelvin,
    report_interval=50000,
    restart_interval=50000,
    restraint="heavy",
)
process = BSS.Process.OpenMM(minimised, protocol, platform="CUDA")
process.start()
process.wait()
equilibrated = process.getSystem()

In [ ]:
BSS.IO.saveMolecules(
    f"outputs/equilibrated_{lig_num}", equilibrated, ["PDB", "RST7", "PRM7"]
)

In [ ]:
BSS.Stream.save(equilibrated, f"outputs/3STD-lig{lig_num}")